## Image Object-Detection Evaluation

This sample shows how use the evaluate a group of models against a given set of metrics for the `image-object-detection` task. 

### Evaluation dataset
We will use the [odfridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip) dataset.

### Model
The goal of evaluating models is to compare their performance on a variety of metrics. `image-object-detection` is a generic task type. As such, the models you pick to compare must be finetuned for the same scenario. Given that we have the dataset, we would like to look for models finetuned for this specific scenario. We will compare `mmd-3x-yolof_r50_c5_8x8_1x_coco` and `mmd-3x-sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco` in this sample, which are available in the `azureml` system registry.

If you'd like to evaluate models that are not in the system registry, you can import those models to your workspace or organization registry and then evaluate them using the approach outlined in this sample. Review the sample notebook for [importing models](../../../import/import_model_into_registry.ipynb).

### Outline
1. Install dependencies
2. Setup pre-requisites
3. Pick the models to evaluate
4. Prepare the dataset for fine-tuning the model
5. Submit the evaluation jobs using the model and data as inputs
6. Review evaluation metrics

### 1. Install dependencies
Before starting off, if you are running the notebook on Azure Machine Learning Studio or running first time locally, you will need the following packages

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

### 2. Setup pre-requisites

#### 2.1 Connect to Azure Machine Learning workspace

Before we dive in the code, you'll need to connect to your workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. `DefaultAzureCredential` should be capable of handling most scenarios. If you want to learn more about other available credentials, go to [set up authentication doc](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk), [azure-identity reference doc](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

Replace `AML_WORKSPACE_NAME`, `RESOURCE_GROUP` and `SUBSCRIPTION_ID` with their respective values in the below cell.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential


experiment_name = (
    "AzureML-Train-Finetune-Vision-OD-Samples"  # can rename to any valid name
)

credential = DefaultAzureCredential()
workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "SUBSCRIPTION_ID"
    resource_group = "RESOURCE_GROUP"
    workspace_name = "AML_WORKSPACE_NAME"

    workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, workspace_name
    )

registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)

#### 2.2 Create compute

In order to finetune a model on Azure Machine Learning studio, you will need to create a compute resource first. **Creating a compute will take 3-4 minutes.** 

For additional references, see [Azure Machine Learning in a Day](https://github.com/Azure/azureml-examples/blob/main/tutorials/azureml-in-a-day/azureml-in-a-day.ipynb). 

In [ ]:
import time
import warnings

from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

model_evaluation_cluster_name = "sample-model-evaluation-compute"

try:
    model_evaluation_compute = workspace_ml_client.compute.get(
        model_evaluation_cluster_name
    )
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    model_evaluation_compute = AmlCompute(
        name=model_evaluation_cluster_name,
        type="amlcompute",
        size="Standard_NC6s_v3",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    workspace_ml_client.begin_create_or_update(model_evaluation_compute).result()

compute_instance_type = model_evaluation_compute.size
print(f"Instance type: {compute_instance_type}")

if compute_instance_type != "STANDARD_NC6S_V3":
    # Print a warning message if compute type is not 'STANDARD_NC6S_V3', i.e. Single GPU V100
    warning_message = (
        "Warning! Currently evaluation is only supported on STANDARD_NC6S_V3 compute type."
        " Please change the compute type to STANDARD_NC6S_V3 if you want to run evaluation."
    )
    warnings.warn(warning_message, category=Warning)
# generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

Below snippet will allow us to query number of GPU's present on the compute. We can use it to set `gpu_per_node` to ensure utilization of all GPUs in the node.

In [ ]:
# This is the number of GPUs in a single node of the selected 'vm_size' compute.
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 1  # default value
gpu_count_found = False
ws_computes = workspace_ml_client.compute.list_sizes()
for ws_compute in ws_computes:
    if ws_compute.name.lower() == model_evaluation_compute.size.lower():
        gpus_per_node = ws_compute.gpus
        print(f"Number of GPUs in compute {ws_compute.name} are {ws_compute.gpus}")
# if gpu_count_found not found, then print an error
if gpus_per_node > 0:
    gpu_count_found = True
else:
    gpu_count_found = False
    print(
        f"No GPUs found in compute. Number of GPUs in compute {model_evaluation_compute.size} 0."
    )

### 3. Pick the models to evaluate

You can evaluate the pretrained models on the repective datasets. Verify that the models selected for evaluation are available in system registry using the below code.

In [ ]:
registry_models = [
    {"name": "mmd-3x-yolof_r50_c5_8x8_1x_coco"},
    {"name": "mmd-3x-sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco"},
]
for model in registry_models:
    all_models = registry_ml_client.models.list(model["name"])
    latest_model = max(all_models, key=lambda x: int(x.version))
    print(latest_model.id)

In this demo notebook, we are using fridge objects dataset. Due to the differences in the labels of the dataset used for pretrained models and that of fridge object dataset, the pretrained models can't be evalauted on the fridge dataset.

Therefore, for the scope of this notebook, we request you to finetune model(s) for fridge objects dataset using [mmdetection-fridgeobjects-object-detection.ipynb](../../finetune/image-object-detection/mmdetection-fridgeobjects-object-detection.ipynb). To finetune and register the model(s), you need to run the above notebook upto "7. Register the fine tuned model with the workspace"  Once you have finetuned and registered the model(s) using above notebook, you can proceed further. 
- Replace `REGISTERED_MODEL_NAME_1/REGISTERED_MODEL_NAME_2` and `REGISTERED_MODEL_VERSION_1/REGISTERED_MODEL_VERSION_2` with that of the registered models from above.

In [ ]:
finetuned_registered_models = [
    {"name": "REGISTERED_MODEL_NAME_1", "version": "REGISTERED_MODEL_VERSION_1"},
    {"name": "REGISTERED_MODEL_NAME_2", "version": "REGISTERED_MODEL_VERSION_2"},
]

for model in finetuned_registered_models:
    model = workspace_ml_client.models.get(model["name"], version=model["version"])
    print(model.id)

### 4. Prepare the dataset for fine-tuning the model

We will use the [odfridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip), a toy dataset called Fridge Objects, which consists of 128 images of 4 labels of beverage container {`can`, `carton`, `milk bottle`, `water bottle`} photos taken on different backgrounds.

All images in this notebook are hosted in [this repository](https://github.com/microsoft/computervision-recipes) and are made available under the [MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE).

#### 4.1 Download the Data
We first download and unzip the data locally. By default, the data would be downloaded in `./data` folder in current directory. 
If you prefer to download the data at a different location, update it in `dataset_parent_dir = ...` in the following cell.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# Create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# Download data
download_url = "https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]

# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# Extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")

# Delete zip file
os.remove(data_file)

In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "images", "31.jpg")
Image(filename=sample_image)

#### 4.2 Upload the images to Datastore through an AML Data asset (URI Folder)

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path=dataset_dir,
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images Object detection",
    name="fridge-items-images-object-detection",
)

uri_folder_data_asset = workspace_ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

#### 4.3 Convert the downloaded data to JSONL

In this example, the fridge object dataset is annotated in Pascal VOC format, where each image corresponds to an xml file. Each xml file contains information on where its corresponding image file is located and also contains information about the bounding boxes and the object labels. 

For documentation on preparing the datasets beyond this notebook, please refer to the [documentation on how to prepare datasets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-prepare-datasets-for-automl-images).


In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. The following script is creating two `.jsonl` files (one for training and one for validation) in the corresponding MLTable folder. In this example, 20% of the data is kept for validation. For further details on jsonl file used for image object detection task in automated ml, please refer to the [data schema documentation for image object-detection task](https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema#object-detection).

In [ ]:
!pip install pycocotools
!pip install simplification
!pip install scikit-image

In [ ]:
import sys

sys.path.insert(0, "../../../../jobs/automl-standalone-jobs/jsonl-conversion/")
from base_jsonl_converter import write_json_lines
from voc_jsonl_converter import VOCJSONLConverter

base_url = os.path.join(uri_folder_data_asset.path, "images/")
converter = VOCJSONLConverter(base_url, os.path.join(dataset_dir, "annotations"))
jsonl_annotations = os.path.join(dataset_dir, "annotations_voc.jsonl")
write_json_lines(converter, jsonl_annotations)

In [ ]:
# If you want to try with a dataset in COCO format, the scripts below shows how to convert it to `jsonl` format. The file "odFridgeObjects_coco.json" consists of annotation information for the `odFridgeObjects` dataset.

# import sys
# sys.path.insert(0, "../../../../jobs/automl-standalone-jobs/jsonl-conversion/")
# from base_jsonl_converter import write_json_lines
# from coco_jsonl_converter import COCOJSONLConverter

# base_url = os.path.join(uri_folder_data_asset.path, "images/")
# print(base_url)
# converter = COCOJSONLConverter(base_url, "./odFridgeObjects_coco.json")
# jsonl_annotations = os.path.join(dataset_dir, "annotations_coco.jsonl")
# write_json_lines(converter, jsonl_annotations)

#### Now split the annotations into train and validation

In [ ]:
import os

# We'll copy each JSONL file within its related MLTable folder
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

# First, let's create the folders if they don't exist
os.makedirs(validation_mltable_path, exist_ok=True)

train_validation_ratio = 5

# Path to the validation files
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

with open(jsonl_annotations, "r") as annot_f:
    json_lines = annot_f.readlines()

index = 0
with open(validation_annotations_file, "w") as validation_f:
    for json_line in json_lines:
        if index % train_validation_ratio == 0:
            # validation annotation
            validation_f.write(json_line)
        else:
            # train annotation
            pass
        index += 1

#### 4.4 Create MLTable data input

Create MLTable data input using the jsonl files created above.

For documentation on creating your own MLTable assets for jobs beyond this notebook, please refer to below resources
- [MLTable YAML Schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable) - covers how to write MLTable YAML, which is required for each MLTable asset.
- [Create MLTable data asset](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK#create-a-mltable-data-asset) - covers how to create MLTable data asset. 

In [ ]:
def create_ml_table_file(filename):
    """Create ML Table definition"""

    return (
        "paths:\n"
        "  - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)


def save_ml_table_file(output_path, mltable_file_contents):
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


# Create and save validation mltable
validation_mltable_file_contents = create_ml_table_file(
    os.path.basename(validation_annotations_file)
)
save_ml_table_file(validation_mltable_path, validation_mltable_file_contents)

### 5. Submit the evaluation jobs using the model and data as inputs
 
Create the job that uses the `model_evaluation_pipeline` component. We will submit one job per model. 

Note that the metrics that the evaluation jobs need to calculate are specified in the [fridge-eval-config.json](./fridge-eval-config.json) file.

All supported evaluation configurations for `image-object-detection` can be found in [README](./README.md).

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(
    name="model_evaluation_pipeline", label="latest"
)

# define the pipeline job
@pipeline()
def evaluation_pipeline(mlflow_model):
    evaluation_job = pipeline_component_func(
        # specify the foundation model available in the azureml system registry or a model from the workspace
        # mlflow_model = Input(type=AssetTypes.MLFLOW_MODEL, path=f"{mlflow_model_path}"),
        mlflow_model=mlflow_model,
        # test data
        test_data=Input(type=AssetTypes.MLTABLE, path=validation_mltable_path),
        # The following parameters map to the dataset fields
        label_column_name="label",
        input_column_names="image_url",
        # Evaluation settings
        task="image-object-detection",
        # config file containing the details of evaluation metrics to calculate
        evaluation_config=Input(
            type=AssetTypes.URI_FILE, path="./fridge-eval-config.json"
        ),
        # config cluster/device job is running on
        # set device to GPU/CPU on basis if GPU count was found
        compute_name=model_evaluation_cluster_name,
        instance_type=compute_instance_type,
        device="gpu" if gpu_count_found else "cpu",
    )
    return {"evaluation_result": evaluation_job.outputs.evaluation_result}

Submit the jobs, passing the model as a parameter to the pipeline created in the above step.

In [ ]:
# submit the pipeline job for each model that we want to evaluate
# you could consider submitting the pipeline jobs in parallel, provided your cluster has multiple nodes
pipeline_jobs = []

for model in finetuned_registered_models:

    # # For each model, fetch the model object from the registry
    # model_object = registry_ml_client.models.get(
    #     model["name"], version=model["version"]
    # )

    # Fetch the model from workspace
    model_object = workspace_ml_client.models.get(
        model["name"], version=model["version"]
    )

    pipeline_object = evaluation_pipeline(
        mlflow_model=Input(type=AssetTypes.MLFLOW_MODEL, path=f"{model_object.id}"),
    )
    # don't reuse cached results from previous jobs
    pipeline_object.settings.force_rerun = True
    pipeline_object.settings.default_compute = model_evaluation_cluster_name
    pipeline_object.display_name = f"eval-{model['name']}-{timestamp}"
    pipeline_job = workspace_ml_client.jobs.create_or_update(
        pipeline_object, experiment_name=experiment_name
    )
    # add model['name'] and pipeline_job.name as key value pairs to a dictionary
    pipeline_jobs.append({"model_name": model["name"], "job_name": pipeline_job.name})
    # wait for the pipeline job to complete
    workspace_ml_client.jobs.stream(pipeline_job.name)

### 6. Review evaluation metrics
Viewing the job in AzureML studio is the best way to analyze logs, metrics and outputs of jobs. You can create custom charts and compare metics across different jobs. See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics?tabs=interactive#view-jobsruns-information-in-the-studio to learn more. 

However, we may need to access and review metrics programmatically for which we will use MLflow, which is the recommended client for logging and querying metrics.

##### 6.1 Initialize MLFlow Client

The models and artifacts that are produced by Azure ML can be accessed via the MLFlow interface.
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

IMPORTANT - You need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow
    pip install mlflow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = workspace_ml_client.workspaces.get(
    name=workspace_ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print(f"\nCurrent tracking uri: {mlflow.get_tracking_uri()}")

from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

##### 6.2 Get the evaluation metrics

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame()

for job in pipeline_jobs:
    # Concat 'tags.mlflow.rootRunId=' and pipeline_job.name in single quotes as filter variable
    filter = "tags.mlflow.rootRunId='" + job["job_name"] + "'"
    runs = mlflow.search_runs(
        experiment_names=[experiment_name], filter_string=filter, output_format="list"
    )

    # Get the training and evaluation runs.
    for run in runs:
        # else, check if run.data.metrics.accuracy exists
        if "mean_average_precision" in run.data.metrics:
            # get the metrics from the mlflow run
            run_metric = run.data.metrics
            # add the model name to the run_metric dictionary
            run_metric["model_name"] = job["model_name"]
            # convert the run_metric dictionary to a pandas dataframe
            temp_df = pd.DataFrame(run_metric, index=[0])
            # concat the temp_df to the metrics_df
            metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)

# move the model_name columns to the first column
cols = metrics_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_df = metrics_df[cols]
metrics_df.head()